## Evaluate MoBa w. pheno
Read MoBa phenotypic information and correlate with polygenic scores computed using different methods. 
This file is only an example written for height as phenotype and should be adapted for other phenotypes.

In [ ]:
%matplotlib inline

In [ ]:
# import packages
import os
import pandas as pd
import numpy as np
from pyreadstat import read_sav
import matplotlib.pyplot as plt
import yaml
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns

In [ ]:
def standardize_resid(df, x, cont_variables=[], cat_variables=[]):
    '''Helper function to return standardized dataframe column valuables
    residualized on other columns with continuous and categorical variables

    Parameters
    ----------
    df: DataFrame
        pandas.DataFrame object
    x: str
        column name of data to residualize
    cont_variables: list of str
        list of column names with continuous variables
    cat_variables: list of str
        list of column names with categorical variables
    '''
    # deal with weirdly named columns
    dataframe = df[[x] + cont_variables + cat_variables].copy()
    assert 'y' not in dataframe.columns, 'can not deal with columns named "y"'
    dataframe['y'] = dataframe[x]

    formula = f'y~'
    if len(cont_variables) > 0:
        formula += '+'.join(cont_variables)
        if len(cat_variables) > 0:
            formula += '+'
    if len(cat_variables) > 0:
        formula += '+'.join([f'C({var})' for var in cat_variables])
    model = smf.ols(formula, data=sm.add_constant(dataframe[['y'] + cont_variables + cat_variables])).fit()
    return model.get_influence().resid_studentized_internal

In [ ]:
# Questionnaire, INFO, pedigree, covariance files
file_info = '/ess/p697/data/durable/phenotypes/mobaQ/PDB2445_MoBa_V12/PDB2445_SV_INFO_V12.sav'
file_Q1 = '/ess/p697/data/durable/phenotypes/mobaQ/PDB2445_MoBa_V12/PDB2445_Q1_v12.sav'
file_Far2 = '/ess/p697/data/durable/phenotypes/mobaQ/PDB2445_MoBa_V12/PDB2445_Far2_v12.sav'
file_pedigree = '/ess/p697/data/durable/projects/moba_qc_imputation/resources/genotyped_pedigree.txt'
file_covariance = '/tsd/p697/data/durable/genotype/MoBaPsychGen_v1/MoBaPsychGen_v1-ec-eur-batch-basic-qc-cov.txt'


In [ ]:
# INFO file with pregnancy ID, mother ID, father ID
info, info_meta = read_sav(file_info)
info

In [ ]:
# preg ID, mother height, father height
mother, mother_meta = read_sav(file_Q1, usecols=['PREG_ID_2445', 'AA87', 'AA88'])
mother = mother.merge(info, on='PREG_ID_2445')
mother

In [ ]:
# father ID, father height
father, father_meta = read_sav(file_Far2, usecols=['F_ID_2445', 'G__5'])  
father

In [ ]:
# make df with height values
height = pd.concat(
    (mother.rename(columns={'M_ID_2445': 'ID_2445', 'AA87': 'height_cm'})[['ID_2445', 'height_cm']],
     mother.rename(columns={'F_ID_2445': 'ID_2445', 'AA88': 'height_cm'})[['ID_2445', 'height_cm']].dropna(),
     ))
height.drop_duplicates(inplace=True)
height = pd.DataFrame(height.groupby('ID_2445')['height_cm'].mean()).reset_index()
height = height[height['ID_2445'] != ''].reset_index(drop=True)
# choose father's self-reported height if available
height = pd.concat([father.rename(columns={'F_ID_2445': 'ID_2445', 'G__5': 'height_cm'}), 
                    height]
                    ).drop_duplicates('ID_2445').sort_values('ID_2445')
# get genotyped pedigree
height = height.merge(pd.read_csv(file_pedigree, sep='\t', 
                      usecols=['ID_2445', 'SENTRIXID', 'FID', 'IID']),
                      on='ID_2445')
height

In [ ]:
# merge covariates with height values.
cov = pd.read_csv('input/keep.txt', sep='\t', header=None, names=['FID', 'IID'])
cov = cov.merge(pd.read_csv(file_covariance, 
                            sep='\t', 
                            usecols=['ID_2445', 'SENTRIXID', 'FID', 'IID', 'SEX', 'YOB', 'genotyping_batch'] + [f'PC{i}' for i in range(1, 11)],
                            ),
                on=['FID', 'IID'])

cov = cov.merge(height, on=['ID_2445', 'IID', 'SENTRIXID'])  # NOTE: omitting FID as they are mismatched
cov.dropna(axis=0, inplace=True)
cov

In [ ]:
# get config for GenoPred
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)
config

In [ ]:
target_list = pd.read_csv(config['target_list'], sep=' ')
target_list

In [ ]:
gwas_list = pd.read_csv(config['gwas_list'], sep=' ')
gwas_list

In [ ]:
# Evaluate PGS vs phenotype, basic test of R2 between residualized PGS genotyped batch, 
# PC1-10 and reported phenotype residualized on SEX, YOB, 
# NOTE: not to be considered rigorous testing, just to demonstrate correlation between 
# phenotype and corresponding PGS.
trait = 'height_cm'
trait_res = 'height_res'
#residualized height
cov[trait_res] = standardize_resid(cov, trait, ['YOB'], ['SEX'])

# create metrics df
metrics = pd.DataFrame({'method': [], 'pgs_col': [], 'R2': []})
for method in config['pgs_methods']:
    file_pgs = os.path.join(config['outdir'], target_list.name[0], 'pgs', gwas_list.population[0], method, gwas_list.name[0],
                             f'{target_list.name[0]}-{gwas_list.name[0]}-{gwas_list.population[0]}.profiles')
    columns = pd.read_csv(file_pgs, sep=' ', nrows=0).columns
    # evaluate all prediction columns, find best fit using LM, showing result
    pgs_columns = columns.drop(['IID', 'FID'])
    for pgs_col in pgs_columns:
        pgs = pd.read_csv(file_pgs, sep=' ', usecols=['IID', pgs_col])
        pgs = pgs.merge(cov[['IID', trait_res, 'genotyping_batch'] + [f'PC{i}' for i in range(1, 11)]], on='IID')
        pgs.dropna(inplace=True, axis=0)
        if pgs.shape[0] == 0:
            continue  # skip if all rows are dropped in case of NaNs
        else:
            pgs['pgs_res'] = standardize_resid(pgs, pgs_col, [f'PC{i}' for i in range(1, 11)], ['genotyping_batch'])
            model_fit = smf.ols(f'{trait_res} ~ pgs_res', data=pgs[[trait_res, 'pgs_res']]).fit()

            metrics = pd.concat((metrics, pd.DataFrame({'method': [method], 'pgs_col': [pgs_col], 'R2': [model_fit.rsquared]})))
metrics.reset_index(inplace=True, drop=True)

In [ ]:
# compare range of R2 values across thresholds etc.
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
sns.swarmplot(data=metrics, x='R2', hue='method', ax=ax)

In [ ]:
# plot score vs trait values (residualized), 
# Shows score columns maximizing R2 for each method. 
nrows = 2
ncols = 4
fig, axes = plt.subplots(nrows, ncols, figsize=(4 * ncols, 4 * nrows), sharey=True, sharex=True)
for ax, method in zip(axes.flatten(), config['pgs_methods']):
    file_pgs = os.path.join(config['outdir'], target_list.name[0], 'pgs', gwas_list.population[0], method, gwas_list.name[0],
                             f'{target_list.name[0]}-{gwas_list.name[0]}-{gwas_list.population[0]}.profiles')
    # plot result maximizing R2
    mmm = metrics[metrics.method == method].reset_index()
    rowind = np.where(mmm.R2 == mmm.R2.max())[0][0]
    pgs_col = mmm.loc[rowind].pgs_col

    pgs = pd.read_csv(file_pgs, sep=' ', usecols=['IID', pgs_col])
    pgs = pgs.merge(cov[['IID', trait_res, 'genotyping_batch'] + [f'PC{i}' for i in range(1, 11)]], on='IID')
    pgs.dropna(inplace=True, axis=0)
    pgs['pgs_res'] = standardize_resid(pgs, pgs_col, [f'PC{i}' for i in range(1, 11)], ['genotyping_batch'])
  
    sns.regplot(
        data=pgs,
        x=trait_res, y='pgs_res',
        label=f'{pgs_col}\nR2={mmm.R2.max():.3f}',
        marker='.',
        ax=ax
    )

    ax.legend(loc='best')
    ax.set_title(method)
    ax.set_xlabel('height (residualized)', labelpad=0)
    ax.set_ylabel('$\mathrm{PGS}_\mathrm{residualized}$', labelpad=0)

for ax in axes.flatten()[len(config['pgs_methods']):]:
    ax.set_visible(False)
